In [1]:
#imports & init
#!pip install -U sentence-transformers
import re
import pandas as pd
import requests
import httplib2
import csv
import string
import IPython
import numpy as np
import webbrowser
import io
import urllib.request
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from bs4.element import Tag
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from PIL import ImageTk, Image
from io import BytesIO
from urllib.request import urlopen
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

In [2]:
#tkinter init
%gui tk
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import ttk

In [3]:
#database
#gives me a list of recipes with ingredients from online csv
matching_recipes = []
url = 'https://raw.githubusercontent.com/jmaio22/PS4/main/food3.csv?token=AOFHMS2UUPD55NB4WASWK73BTCKMK'
df = pd.read_csv(url,engine='python', sep=',',quotechar='"')
df = df[['food']]
df['food'] = df['food'].str.lower()
igens = df['food'].values.tolist()
#create sublist of each recipe in a single list
indices = []
for i, elem in enumerate(igens):
    if 'https:' in elem:
        indices.append(i)
#getting indexes of each link to delimit the data
x = -1
x1 = 0
recipedata = []
for items in indices:
  recipedata.append(list(igens[x+1:items+1]))
  x = items
#len(recipedata)
#recipedata[762]

In [4]:
recipedata[762]

['all-purpose flour',
 'puff pastry',
 'cream cheese',
 'raisins',
 'cinnamon',
 'sugar',
 'https://www.seriouseats.com/cinnamon-raisin-puff-pastry-waffle-recipe']

In [5]:
#scraping title of recipe
def get_title(links):
    page = requests.get(links)
    soup = BeautifulSoup(page.content, "html.parser")
    results = (str(soup.find(id="heading_1-0"))).split('\n')
    title = re.sub("[\(\<].*?[\)\>]", "", results[1])
    return title

In [6]:
#scraping image of recipe
def get_image(links):
    page = requests.get(links)
    soup = BeautifulSoup(page.content, "html.parser")
    results = (str(soup.find(id="figure_2-0"))).split('\n')
    recipe_image = results[1].split('"')
    for items in recipe_image:
        if items[0:3] == "htt":
            recipe_image = items
            break
    return recipe_image

In [7]:
# get_title('https://www.seriouseats.com/homemade-egg-mcmuffin-mcdonalds-recipe')
# get_image('https://www.seriouseats.com/pasta-al-limone')

In [8]:
def open_recipe(event):
    for index,selected_item in enumerate(tree2.selection()):
        global ewe
        global hyplinks
        curItem = tree2.focus()
        ewe = tree2.item(curItem,'text')
        ewe = (str(ewe))
        print(ewe)
        webbrowser.open(hyplinks.get(ewe))
#         print(test)
#         x = test[index]
#         webbrowser.open(x)

In [9]:
from PIL import ImageTk, Image
#searching database for matching recipes from ing_input, returning matching recipes list
test = []
def recipe_search():
    global hyplinks
    hyplinks = {}
    for i in tree2.get_children()[::len(tree2.get_children())-1]:
        tree2.delete(i)
    matching_recipes = []
    ewe = 0
    for recipes in recipedata:
      match_count = 0 
      recipe_embeddings = model.encode(recipes[0:-1])
      input_embeddings = model.encode(ing_input)
      for ings in input_embeddings:
        recipe_embeddings2 = recipe_embeddings 
        for index,ings2 in enumerate(recipe_embeddings2):
          if cosine_similarity(ings.reshape(1, -1),ings2.reshape(1, -1))[0][0] > 0.85:
                match_count += 1
                ings2 = 9859876310513751309753105673153165731653156315679367957960
                break
            
      if match_count >= 0.99*(len(recipes)-1):
        print(recipes[-1])
        matching_recipes.append(recipes[-1])
        test.append(recipes[-1])
        hyplinks[get_title(recipes[-1])] = recipes[-1]
    for items in matching_recipes:
        tree2.insert('', tk.END, text=get_title(items))
        add_img(get_image(items))
    print(hyplinks)


def add_ing(x):
    if entry1.get().isspace() == False:
        if not(entry1.get()) in ing_input:
            test = []
            ing_input.append(entry1.get())
            print(ing_input)
            tree.insert('', tk.END, values=(ing_input[-1],))
            entry1.delete(0, 'end')     
            
def remove_item(event):
    curItem = tree.focus()
    x = tree.item(curItem,'values')
    x = (str(x[0]))
    ing_input.remove(x)
    for index,selected_item in enumerate(tree.selection()):
        tree.delete(selected_item)


    
ing_input = []
win = tk.Tk()
win.geometry('700x600')
win.resizable(False, False)
win.title('Recipe Finder')
win.bind('<Return>', add_ing)
win.configure(background='white')

s = ttk.Style(win)
win.tk.call("source", "sun-valley.tcl")
s.theme_use("sun-valley-light")

entry1 = ttk.Entry (win,width=40) 
entry1.pack()
entry1.place(x=70,y=70)

tree = ttk.Treeview(win, columns='Ingredients', show='headings')
tree.heading('Ingredients', text='Ingredients', anchor=CENTER)
tree.pack()
tree.place(x=370,y=30)
tree.bind('<<TreeviewSelect>>', remove_item)

tree2 = ttk.Treeview(win)
tree2.heading('#0', text='Recipes', anchor=CENTER)
tree2.pack()
tree2.place(x=70,y=190)
tree2.bind('<<TreeviewSelect>>', open_recipe)

button1 = ttk.Button(win,command=recipe_search,width=20,text='Search For Recipes')
button1.pack()
button1.place(x=70,y=120)

exit_button = ttk.Button(win, text="Exit", command=win.destroy)
exit_button.pack(pady=20)
exit_button.place(x=20,y=10)

def add_img(link):
    global img
    global img_data
    global response
    img_url = link
    response = requests.get(img_url)
    img_data = response.content
    img = Image.open(BytesIO(img_data))
    img = img.resize((128,128))
    img = ImageTk.PhotoImage(img)
    tree2.insert('', tk.END, image=img)
    

In [10]:
# https://stackoverflow.com/questions/16996432/how-do-i-bind-the-enter-key-to-a-function-in-tkinter/16996475
# https://stackoverflow.com/questions/2260235/how-to-clear-the-entry-widget-after-a-button-is-pressed-in-tkinter    
# https://stackoverflow.com/questions/31015774/removing-a-selection-from-a-listbox-as-well-as-remove-it-from-the-list-that-pro
# https://www.pythontutorial.net/tkinter/tkinter-listbox/
# https://www.reddit.com/r/Python/comments/lps11c/how_to_make_tkinter_look_modern_how_to_use_themes/
# https://www.pythontutorial.net/tkinter/tkinter-treeview/
# https://riptutorial.com/tkinter/example/31880/treeview--basic-example
# https://stackoverflow.com/questions/49935927/tkinter-treeview-string-with-space-issue
# https://www.kite.com/python/answers/how-to-convert-letters-to-numbers-in-python
# https://stackoverflow.com/questions/34849035/how-to-get-the-value-of-a-selected-treeview-item

['black pepper']
['black pepper', 'salt']
['black pepper', 'salt', 'unsalted butter']
['black pepper', 'salt', 'unsalted butter', 'lemon']
['black pepper', 'salt', 'unsalted butter', 'lemon', 'garlic']
['black pepper', 'salt', 'unsalted butter', 'lemon', 'garlic', 'fresh spaghetti']
['black pepper', 'salt', 'unsalted butter', 'lemon', 'garlic', 'fresh spaghetti', 'parmigiano reggiano cheese']
https://www.seriouseats.com/pasta-al-limone
{'Pasta al Limone Recipe': 'https://www.seriouseats.com/pasta-al-limone'}
Pasta al Limone Recipe
['black pepper', 'salt', 'unsalted butter', 'lemon', 'garlic', 'fresh spaghetti', 'parmigiano reggiano cheese', 'bacon']
['black pepper', 'salt', 'unsalted butter', 'lemon', 'garlic', 'fresh spaghetti', 'parmigiano reggiano cheese', 'bacon', 'spring onions']
['black pepper', 'salt', 'unsalted butter', 'lemon', 'garlic', 'fresh spaghetti', 'parmigiano reggiano cheese', 'bacon', 'spring onions', 'fennel']
['black pepper', 'salt', 'unsalted butter', 'lemon', 'ga